<a href="https://colab.research.google.com/github/rinabuoy/ML/blob/master/Word_Ladder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# RUN THIS ONCE IN THE BEGINNING TO INSTALL PYENCHANT
!pip install pyenchant
!apt-get install libenchant1c2a

     |████████████████████████████████| 61kB 1.9MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common enchant hunspell-en-us
  libaspell15 libhunspell-1.6-0 libtext-iconv-perl
Suggested packages:
  aspell-doc spellutils wordlist hunspell openoffice.org-hunspell
  | openoffice.org-core libenchant-voikko
The following NEW packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common enchant hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
0 upgraded, 10 newly installed, 0 to remove and 11 not upgraded.
Need to get 1,310 kB of archives.
After this operation, 5,353 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libtext-iconv-perl amd64 1.7-5build6 [13.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libaspell15 am

# Successor States

In [2]:
import enchant, string

def successors(state):
  """
  Given a word, find all possible English word results from changing one letter.
  Return a list of (action, word) pairs, where action is the index of the
  changed letter.
  """
  d = enchant.Dict("en_US")
  child_states = []
  for i in range(len(state)):
    new = [state[:i]+x+state[i+1:] for x in string.ascii_lowercase]
    words = [x for x in new if d.check(x) and x != state]
    child_states = child_states + [(i, word) for word in words]
  return child_states

# Node Expansion

In [3]:
from heapq import heappush, heappop

def expand(node):
  """
  Given a node, return a list of successor nodes
  """
  state = node['state']
  children = []
  for successor in successors(state):
    children.append({'state':successor[1], 'parent':node,
                     'action':successor[0], 'cost':node['cost']+1})
  return children


def best_first_search(state, goal, f, depth_limit):
  """
  Inputs: Initial state, goal state, priority function, depth limit
  Returns node containing goal or None if no goal found within depth limit, 
  max frontier size, total nodes expanded
  """
  node = {'state':state, 'parent':None, 'action':None, 'cost':0}
  frontier = []
  heappush(frontier, (f(node, goal), id(node), node))
  reached = {state: node}
  max_frontier = 1
  nodes_expanded = 0

  while frontier:
    max_frontier = max(max_frontier, len(frontier)) 
    node = heappop(frontier)[2]

    if node['state']==goal: 
      return node, max_frontier, nodes_expanded 
    if node['cost']>=depth_limit: 
      continue 
    for c in expand(node): 
      if c['state'] not in reached or c['cost'] <reached[c['state']]['cost']:
        reached[c['state']]=c
        heappush(frontier, (f(c, goal), id(c), c))
    nodes_expanded += 1


  return None, max_frontier, nodes_expanded

# Cost Function

In [4]:
def f_bfs(node, goal):
  return id(node)


def f_dfs(node, goal):
  return -id(node)

# Utils

In [5]:
def sequence(node):
  words = [node['state']]
  while node['parent'] is not None:
    node = node['parent']
    words.insert(0, node['state'])
  return words

def results(solution):
  if solution[0] is not None:
    words = sequence(solution[0])
  else: words = "No solution!"
  print(words)
  print("Total cost:", len(words)-1)
  print("Max frontier size:", solution[1])
  print("Nodes expanded:", solution[2])
  print("")

# Test - Hit -> Cog

In [6]:
start = 'hit'
goal = 'cog'

solution = best_first_search(start, goal, f_bfs, float("inf"))
print("BFS")
results(solution)

solution = best_first_search(start, goal, f_dfs, float("inf"))
print("DFS")
results(solution)

BFS
['hit', 'hot', 'cot', 'cog']
Total cost: 3
Max frontier size: 492
Nodes expanded: 1698

DFS
['hit', 'wit', 'wot', 'wog', 'cog']
Total cost: 4
Max frontier size: 840
Nodes expanded: 536



# Test - Cold -> Warm

In [7]:
start = 'warm'
goal = 'cold'

solution = best_first_search(start, goal, f_bfs, float("inf"))
print("BFS")
results(solution)

solution = best_first_search(start, goal, f_dfs, float("inf"))
print("DFS")
results(solution)

BFS
['warm', 'ware', 'wave', 'gave', 'gape', 'vape', 'vase', 'lase', 'lose', 'lost', 'loot', 'foot', 'fool', 'foll', 'fold', 'cold']
Total cost: 15
Max frontier size: 447
Nodes expanded: 110

DFS
['warm', 'wart', 'want', 'wand', 'rand', 'rank', 'rink', 'link', 'line', 'lane', 'lake', 'make', 'mace', 'pace', 'pale', 'tale', 'tall', 'toll', 'coll', 'cold']
Total cost: 19
Max frontier size: 3577
Nodes expanded: 3118



# A* Search

In [8]:
# Heuristic Function

def f_astar(node, goal):
  # YOUR CODE HERE
  return sum([c!=goal[i] for i,c in enumerate(node)])

In [10]:
start = 'warm'
goal = 'cold'

solution = best_first_search(start, goal, f_astar, float("inf"))
print("A*")
results(solution)


A*
['warm', 'farm', 'firm', 'film', 'fill', 'foll', 'coll', 'cold']
Total cost: 7
Max frontier size: 504
Nodes expanded: 191

